In [ ]:
import uvage

#GLobal Variables ----------------------------------------------------------------
score = 0
game_on = False
game_over = True
game_over_screen = True
player_name = 'z'
scoreboard_values = []

#Set-up Game Objects ----------------------------------------------------------------
camera = uvage.Camera(800, 800)
scoreboard = uvage.from_color(400, 400, 'white', 400, 400)
button = uvage.from_color(600, 600, 'red', 200,200)
score_table = []

#Inter-Session Progress ----------------------------------------------------------------

def store_highscore():

    global player_name, score

    highscore_file = open('highscore.txt', 'a')
    highscore_file.write(player_name + ',' + str(score))
    highscore_file.close()

    return

def clear_highscore():
    highscore_file = open('highscore.txt', 'w')
    print(file = highscore_file)
    highscore_file.close()

    return

def retrieve_highscorelist():

    highscore_file = open('highscore.txt', 'r')
    highscore = []
    
    for row in highscore_file.read().split('\n'):
        highscore.append(row.split(','))

    highscore_file.close()
    
    return highscore.sort(key = lambda x: x[1])

def create_score_table(values):
    
    scoreboard_values = values

    rank = 1
    entries = 0 

    while entries < 10 and entries <= (len(scoreboard_values) - 2):
        text = "{} | Score:{}".format(scoreboard_values[rank][0], scoreboard_values[rank][1])
        entry_diplay = uvage.from_text(scoreboard.x, scoreboard.top - 20 + 40 * rank, text, 40, "black")
        score_table.append(entry_diplay)
        rank += 1
        entries += 1

    return

# Collectibles --------------------------------

# Snake --------------------------------

# Restart -----------------------------------------

def restart_game():
    return

# Game Code ----------------------------------------------------------------

def game():

    global game_over, game_on, game_over_screen

    if game_over == True:
        store_highscore()
        scoreboard_values = retrieve_highscorelist()
        create_score_table(scoreboard_values)
        game_over_screen = True
        game_on == False
        game_over = False
    
    if game_over_screen == True:
        camera.draw(scoreboard)
        for row in score_table:
            camera.draw(row)
        camera.draw(button)
        if button.contains(camera.mouse) and camera.mouseclick == True:
            restart_game()
            game_on = True
            game_over_screen == False

    camera.display()
    return

#Run Game ----------------------------------------------------------------
uvage.timer_loop(30, game)

In [ ]:
store_highscore()